In [9]:
import os
import errno
import numpy as np
import deepcell
from tensorflow.python import keras

In [2]:
from tensorflow.keras.optimizers import SGD
from deepcell.utils.train_utils import rate_scheduler

test_size = .10  # % of data saved as test
receptive_field = 81  # should be adjusted for the scale of the data

optimizer = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

lr_sched = rate_scheduler(lr=0.01, decay=0.99)

# Sample mode settings
batch_size = 64  # number of images per batch (should be 2 ^ n)
win = (receptive_field - 1) // 2  # sample window size
balance_classes = True  # sample each class equally

In [6]:
base_name = 'Training_Freeze_1'
npz_name = '/data/npz_data/' + base_name + ".npz"
MODEL_DIR = '/data/models/' + '20190822_training_freeze_1'
fgbg_model_name = base_name + "_fgbg_81_rf_256_dense_64_conv_subsampled"

if not os.path.isdir(MODEL_DIR):
    os.makedirs(MODEL_DIR)

LOG_DIR = '/data/logs'

### Load the Training Data

In [4]:
CHANNEL_AXIS = 3
training_data = np.load(npz_name)

X, y = training_data["X"], training_data["y"]
print("X.shape: {} & y.shape: {}".format(X.shape, y.shape))

X.shape: (4, 1024, 1024, 5) & y.shape: (4, 1024, 1024, 1)


#### Instantiate the fgbg model

In [5]:
from deepcell import model_zoo

fgbg_model = model_zoo.bn_feature_net_2D(
    receptive_field=receptive_field,
    n_channels=X.shape[CHANNEL_AXIS],
    n_features=2)

In [7]:
fgbg_model_name

'Training_Freeze_1_fgbg_81_rf_256_dense_64_conv_subsampled'

#### Train the fgbg model

In [ ]:
from deepcell.training import train_model_sample

fgbg_model = train_model_sample(
    model=fgbg_model,
    dataset=npz_name,
    model_name=fgbg_model_name,
    test_size=test_size,
    optimizer=optimizer,
    window_size=(win, win),
    batch_size=128,
    transform='fgbg',
    n_epoch=24,
    balance_classes=balance_classes,
    max_class_samples=400000,
    model_dir=MODEL_DIR,
    log_dir=LOG_DIR,
    lr_sched=lr_sched,
    rotation_range=180,
    flip=True,
    shear=False,
    zoom_range=(0.8, 1.2),
    val_monitor=False,
    save_period=1)

using all data as training data
Using class weights of {0: 1.0, 1: 1.0}
X_train shape: (4, 1024, 1024, 5)
y_train shape: (4, 1024, 1024, 1)
Output Shape: (None, 2)
Number of Classes: 2
Training on 1 GPUs
the max_class_samples per image is 100000
analyzing image 0
the least represented class has 91377 examples
analyzing class 0
downsampling from 91377 examples per class
analyzing class 1
downsampling from 797872 examples per class
analyzing image 1
the least represented class has 299309 examples
max_class_samples is less than the smalleset class, downsampling all classes
analyzing class 0
downsampling from 299309 examples per class
analyzing class 1
downsampling from 589940 examples per class
analyzing image 2
the least represented class has 267420 examples
max_class_samples is less than the smalleset class, downsampling all classes
analyzing class 0
downsampling from 267420 examples per class
analyzing class 1
downsampling from 621829 examples per class
analyzing image 3
the least repr